![autogluon](https://auto.gluon.ai/stable/_static/AutogluonLogo.png)


AutoGluon enables easy-to-use and easy-to-extend AutoML with a focus on automated stack ensembling, deep learning, and real-world applications spanning text, image, and tabular data. Intended for both ML beginners and experts, AutoGluon enables you to:

Quickly prototype deep learning and classical ML solutions for your raw data with a few lines of code.

Automatically utilize state-of-the-art techniques (where appropriate) without expert knowledge.

Leverage automatic hyperparameter tuning, model selection/ensembling, architecture search, and data processing.

Easily improve/tune your bespoke models and data pipelines, or customize AutoGluon for your use-case.

In [ ]:
!pip install "autogluon.tabular[all]==0.1.1b20210312"
!pip install AutoViz
!pip install xlrd

In [ ]:
import pandas as pd
dfe = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')

from autoviz.AutoViz_Class import AutoViz_Class

#Instantiate the AutoViz class
AV = AutoViz_Class()

In [ ]:
dftc = AV.AutoViz(filename='', 
                  sep ='' , 
                  depVar ='Survived', 
                  dfte = dfe, 
                  header = 0, 
                  verbose = 0, 
                  lowess = False, 
                  chart_format ='png', 
                  max_rows_analyzed = 100000, 
                  max_cols_analyzed = 30
)

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.tabular.models.lr.lr_rapids_model import LinearRapidsModel
from autogluon.tabular.models.knn.knn_rapids_model import KNNRapidsModel

train_data = TabularDataset('../input/tabular-playground-series-apr-2021/train.csv')
test_data = TabularDataset('../input/tabular-playground-series-apr-2021/test.csv')

label = 'Survived'

In [ ]:
#using integrated RAPIDS models along with some boosting models
predictor = TabularPredictor(
    label=label,
    eval_metric='accuracy',
    learner_kwargs={'ignored_columns': ['PassengerId']}
).fit(
    train_data,
    presets='best_quality',
    hyperparameters={'XGB': {'ag_args_fit': {'num_gpus': 1}},
        'GBM': [{}, {'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, 'GBMLarge'],
        'NN': {'ag_args_fit': {'num_gpus': 1}},
        'CAT': {'ag_args_fit': {'num_gpus': 1}},
        KNNRapidsModel: {},
        LinearRapidsModel: {},
       
    },
)

In [ ]:
import pandas as pd
submission = test_data[['PassengerId']]
test_pred = predictor.predict(test_data)
submission = pd.concat([submission, test_pred], axis=1)
submission.to_csv('submission.csv', index=False)
submission.head()

In [ ]:
predictor.leaderboard(silent=True)